In [1]:
import numpy as np
import pandas as pd
import simpeg
import libaarhusxyz
import dill

work_dir ='../data/'
df_em_inv = pd.read_parquet(work_dir+"gmd_4_inv.parquet")
df_em_syn = pd.read_parquet(work_dir+"gmd_4_syn.parquet")
meta_data_dict = dill.load(open(work_dir+"gmd_4_meta_data.pik", "rb"))

i_start_ch1 = 0
i_start_ch2 = 0

line = df_em_inv['line_no'].values[:]
topography = df_em_inv[['utmx', 'utmy', 'elevation']].values[:, :]
source_heights = df_em_inv['altitude_[m]'].values[:]
data_ch1 = df_em_inv[meta_data_dict['ch1_names']].values[:, i_start_ch1:]
data_ch2 = df_em_inv[meta_data_dict['ch2_names']].values[:, i_start_ch2:]
data_std_ch1 = df_em_inv[meta_data_dict['std_ch1_names']].values[:, i_start_ch1:] - 1
data_std_ch2 = df_em_inv[meta_data_dict['std_ch2_names']].values[:, i_start_ch2:] - 1
n_sounding = len(line)
times_ch1 = meta_data_dict['ch1_times'][i_start_ch1:]
times_ch2 = meta_data_dict['ch2_times'][i_start_ch2:]

system = libaarhusxyz.parse_gex("../data/gex/20240724_20078_Kansas_AGF_304_XYZ.GEX")
area = system['General']['TxLoopArea']
waveform_ch2 = system['General']['WaveformHMPoint']
waveform_ch1 = system['General']['WaveformLMPoint']
time_input_currents_ch2 = waveform_ch2[:,0]
input_currents_ch2 = waveform_ch2[:,1]
time_input_currents_ch1 = waveform_ch1[:,0]
input_currents_ch1 = waveform_ch1[:,1]

thickness = np.array([ 2.   ,  2.129,  2.267,  2.413,  2.569,  2.736,  2.912,  3.101,
        3.301,  3.514,  3.741,  3.983,  4.241,  4.515,  4.807,  5.117,
        5.448,  5.8  ,  6.175,  6.574,  6.999,  7.451,  7.933,  8.446,
        8.992,  9.573, 10.19 , 10.85 , 11.55 , 12.3  , 13.09 , 13.94 ,
       14.84 , 15.8  , 16.82 , 17.91 , 19.06 , 20.3  , 21.61 ])

input_data_dict = {
    "topography": topography,
    "source_heights": source_heights,
    "thickness": thickness,
    "time_input_currents_ch2":time_input_currents_ch2,
    "time_input_currents_ch1":time_input_currents_ch1,
    "input_currents_ch2":input_currents_ch2,
    "input_currents_ch1":input_currents_ch1,    
    "times_ch1":times_ch1,    
    "times_ch2":times_ch2,      
    "bw_cutoff_frequency_ch1":system['Channel1']['TiBLowPassFilter'][1],
    "bw_power_ch1":system['Channel1']['TiBLowPassFilter'][0],
    "bw_cutoff_frequency_ch2":system['Channel2']['TiBLowPassFilter'][1],
    "bw_power_ch2":system['Channel2']['TiBLowPassFilter'][0],    
    "lp_cutoff_frequency_ch1": system['General']['RxCoilLPFilter'][1],
    "lp_power_ch1": system['General']['RxCoilLPFilter'][0],    
    "lp_cutoff_frequency_ch2": system['General']['RxCoilLPFilter'][1],
    "lp_power_ch2": system['General']['RxCoilLPFilter'][0],        
    "rx_coil_position": system['General']['RxCoilPosition'],
    "data_ch1":data_ch1,
    "data_ch2":data_ch2,
    "data_std_ch1":data_std_ch1,
    "data_std_ch2":data_std_ch2,
    
}

dill.dump(input_data_dict, open("input_gmd_4.pik", "wb"))

header [General] parsed
header [Channel1] parsed
header [Channel2] parsed
